# Dependencies

In [1]:
pip install fastapi uvicorn pydantic transformers!


[notice] A new release of pip is available: 24.1.2 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


# Imports

In [2]:
from fastapi import FastAPI, HTTPException
from pydantic import BaseModel
from typing import Optional
import transformers
import uvicorn
import nest_asyncio

In [3]:
nest_asyncio.apply()

In [4]:
# data model for content verification request

class ContentVerificationRequest(BaseModel):
    content: str

In [5]:
# fast api app
app = FastAPI()

In [6]:
model_name = "distilbert-base-uncased"  # Example model
tokenizer = transformers.AutoTokenizer.from_pretrained(model_name)
model = transformers.AutoModelForSequenceClassification.from_pretrained(model_name)

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

C:\Users\haoha\AppData\Local\Programs\Python\Python312\Lib\site-packages\huggingface_hub\file_download.py:159: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\haoha\.cache\huggingface\hub\models--distilbert-base-uncased. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [10]:
@app.get("/")
def read_root():
    return {"message": "Content Verification API is running"}

In [11]:
@app.post("/api/verify")
async def verify_content(request: ContentVerificationRequest):
    try:
        content = request.content
        inputs = tokenizer(content, return_tensors='pt')
        outputs = model(**inputs)

        # For demonstration, just return a fake result
        result = {
            "score": outputs.logits.softmax(dim=1).tolist(),
            "label": "fake_label"  # Replace with actual label prediction logic
        }
        return {"status": "success", "result": result}
    except Exception as e:
        raise HTTPException(status_code=500, detail=f"An error occurred: {str(e)}")